In [1]:
import lxml.etree
p = lxml.etree.parse('/Users/alehmann/Documents/Documents/HIH/phd ansøgning/Trafikvej2011_Årsdøgntrafik2010.kml')

ImportError: No module named lxml.etree

In [4]:
kml = 'http://www.opengis.net/kml/2.2'

In [5]:
trafik = []
for place in p.findall('//{{{0}}}Placemark'.format(kml)):
    talt = place.findall('{{{0}}}ExtendedData/{{{0}}}SchemaData/{{{0}}}SimpleData[@name="rsdgntrafik_talt"]'.format(kml))[0]
    skoen = place.findall('{{{0}}}ExtendedData/{{{0}}}SchemaData/{{{0}}}SimpleData[@name="rsdgntrafik_skn"]'.format(kml))[0]
    trafik.append(int(talt.text)+int(skoen.text))
    if skoen.text == "50" : print place.findall('{{{0}}}ExtendedData/{{{0}}}SchemaData/{{{0}}}SimpleData[@name="Lb_nr"]'.format(kml))[0].text

608


In [6]:
max_trafik = max(trafik)

In [7]:
min_trafik = min(trafik)

In [8]:
from math import atan2

def rad(angle):
    return angle/180*pi

def haversine(*args):
	# a = sin²(Δφ/2) + cos(φ1).cos(φ2).sin²(Δλ/2)
    # c = 2.atan2(√a, √(1−a))
    # d = R.c
    R = 6371
    lat, lon, lat1, lon1 = map(rad, args)
    dlat = lat1 - lat
    dlon = lon1 - lon
    a = sin(dlat/2)**2 + cos(lat)*cos(lat1)*sin(dlon/2)**2
    c = 2*atan2(sqrt(a),sqrt(1-a))
    d = R*c
    return d

In [9]:
total = 0
dups = set()
for place in p.findall('//{{{0}}}Placemark'.format(kml)):
    line = place.findall('{{{0}}}Style/{{{0}}}LineStyle'.format(kml))[0]
    color = line.find('{{{0}}}color'.format(kml))
    talt = place.findall('{{{0}}}ExtendedData/{{{0}}}SchemaData/{{{0}}}SimpleData[@name="rsdgntrafik_talt"]'.format(kml))[0]
    skoen = place.findall('{{{0}}}ExtendedData/{{{0}}}SchemaData/{{{0}}}SimpleData[@name="rsdgntrafik_skn"]'.format(kml))[0]
    serialnumber = place.findall('{{{0}}}ExtendedData/{{{0}}}SchemaData/{{{0}}}SimpleData[@name="Lb_nr"]'.format(kml))[0]
    if serialnumber.text not in dups:
        dups.add(serialnumber.text)
    else:
        print serialnumber.text,"<<<<"
    amount = int(talt.text)+int(skoen.text)  
    width = lxml.etree.SubElement(line,'{{{0}}}width'.format(kml))
    width.text = str(int (10.*amount/max_trafik +1))
    if int(skoen.text): color.text = 'ff00ff00'
    pp = False    
    linecoord = place.findall('{{{0}}}LineString/{{{0}}}coordinates'.format(kml))
    if not linecoord:

#        print serialnumber.text,
        pp = True
        linecoord = place.findall('{{{0}}}MultiGeometry/{{{0}}}LineString/{{{0}}}coordinates'.format(kml))
    dist = 0
    for lin in linecoord:
        coord = [tuple(map(float,x.split(','))) for x in lin.text.split(' ') ]
        lon0,lat0 = coord[0]

        for lon,lat in coord[1:]:
            dist += haversine(lat0,lon0,lat,lon)
            lat0,lon0 = lat,lon
        if serialnumber.text == "1090": print coord, dist

    if dist < 0.01: print dist,serialnumber.text
    pp = False
    total += dist
    length = lxml.etree.SubElement(place,'{{{0}}}Length'.format(kml))
    length.text = str(dist)
print total,len(p.findall('//{{{0}}}Placemark'.format(kml)))

[(9.999962383154033, 56.152072949135125), (9.998971803378318, 56.151866166726826), (9.99869424210255, 56.15183041015547)] 0.0831629244465
1090 <<<<
[(9.974727498107958, 56.150676406444404), (9.974727365397174, 56.150676407491105)] 8.22050163055e-06
8.22050163055e-06 1090
636.091292493 1006


In [11]:
p.write('/Users/alehmann/Desktop/trafik2.kml')